In [6]:
import cv2
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow.keras as K
import tensorflow.keras.backend as Kback
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf

# from google.colab import drive
# drive.mount('/content/drive')

# Model Training

In [7]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = Kback.sum(Kback.round(Kback.clip(y_true * y_pred, 0, 1)))
    possible_positives = Kback.sum(Kback.round(Kback.clip(y_true, 0, 1)))
    predicted_positives = Kback.sum(Kback.round(Kback.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + Kback.epsilon())
    recall = true_positives / (possible_positives + Kback.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+Kback.epsilon())
    return f1_val

METRICS = [
      "accuracy",
      K.metrics.Precision(name='precision'),
      K.metrics.Recall(name='recall'),
      K.metrics.AUC(name='auc'),
      f1_score
]

In [8]:
input_layer = K.Input(shape=(160,160,3))
resnet_model = K.applications.EfficientNetB0(include_top = False, weights = "imagenet", input_tensor = input_layer)
for layer in resnet_model.layers:
    layer.trainable = False
for i, layer in enumerate(resnet_model.layers):
    print(i, layer.name, "-", layer.trainable)

0 input_2 - False
1 rescaling_1 - False
2 normalization_1 - False
3 tf.math.truediv_1 - False
4 stem_conv_pad - False
5 stem_conv - False
6 stem_bn - False
7 stem_activation - False
8 block1a_dwconv - False
9 block1a_bn - False
10 block1a_activation - False
11 block1a_se_squeeze - False
12 block1a_se_reshape - False
13 block1a_se_reduce - False
14 block1a_se_expand - False
15 block1a_se_excite - False
16 block1a_project_conv - False
17 block1a_project_bn - False
18 block2a_expand_conv - False
19 block2a_expand_bn - False
20 block2a_expand_activation - False
21 block2a_dwconv_pad - False
22 block2a_dwconv - False
23 block2a_bn - False
24 block2a_activation - False
25 block2a_se_squeeze - False
26 block2a_se_reshape - False
27 block2a_se_reduce - False
28 block2a_se_expand - False
29 block2a_se_excite - False
30 block2a_project_conv - False
31 block2a_project_bn - False
32 block2b_expand_conv - False
33 block2b_expand_bn - False
34 block2b_expand_activation - False
35 block2b_dwconv - Fa

In [9]:
def fft_2d(feature_map):
    feature_map = tf.cast(feature_map, tf.complex64)
    X1 = tf.signal.fft2d(feature_map)
    X1 = tf.abs(X1)
    return X1   

inputs = K.layers.Input(shape=(160,160,3), name="image") 
resnet50 = resnet_model(inputs)
fft_layer = K.layers.Lambda(lambda x: fft_2d(x))(resnet50)
fft_flat = K.layers.GlobalAveragePooling2D()(fft_layer)
output = K.layers.Dense(20, activation='softmax')(fft_flat)

model = K.Model(inputs=inputs, outputs=fft_flat)
optimizer = K.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model.compile(loss=["categorical_crossentropy"], metrics=METRICS, optimizer = optimizer)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 160, 160, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 5, 5, 1280)       4049571   
                                                                 
 lambda_1 (Lambda)           (None, 5, 5, 1280)        0         
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
Total params: 4,049,571
Trainable params: 0
Non-trainable params: 4,049,571
_________________________________________________________________


In [10]:
import os
import random
import numpy as np

Y_train = []
X_train = []
for i in range(1,21):
  folder = "E:/data/train/"+str(i)+"/"
  for image in os.listdir(os.path.join(folder)):
    I = []
    img = cv2.imread(os.path.join(folder,image))
    img = cv2.resize(img, (160, 160))
    img = img/255
    I.append(img)
    I = np.asarray(I)
    X_train.append(np.asarray(model.predict(I)[0]))
    Y_train.append(i)

Y_train = np.asarray(Y_train)
X_train = np.asarray(X_train)

Y_test = []
X_test = []
for i in range(1,21):
  folder = "E:/data/test/"+str(i)+"/"
  for image in os.listdir(os.path.join(folder)):
    I = []
    img = cv2.imread(os.path.join(folder,image))
    img = cv2.resize(img, (160, 160))
    img = img/255
    I.append(img)
    I = np.asarray(I)
    X_test.append(np.asarray(model.predict(I)[0]))
    Y_test.append(i)

Y_test = np.asarray(Y_test)
X_test = np.asarray(X_test)

shuffler = np.random.permutation(len(X_train))
X_train = X_train[shuffler]
Y_train = Y_train[shuffler]




 

 





#Load model
# model_xgb_2 = xgb.Booster()
# model_xgb_2.load_model("model.json")

1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 42ms/step


...

1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 45ms/step


In [11]:
shuffler = np.random.permutation(len(X_test))
X_test = X_test[shuffler]
Y_test = Y_test[shuffler]

In [13]:
!pip install xgboost

     ---------------------------------------- 89.1/89.1 MB 1.1 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)

In [18]:
#XGBoost as classifier using the features of CNN+FFT
import xgboost as xgb
my_model = xgb.XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
my_model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [19]:
# Predicting the Test set results
y_pred = my_model.predict(X_test)

In [20]:
# Making the Confusion Matrix
cm = confusion_matrix(Y_test, y_pred)
disp = ConfusionMatrixDisplay(cm,display_labels=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])

In [41]:
Y_test = le.fit_transform(Y_test)

In [42]:
#Accuracy
from sklearn.metrics import accuracy_score
print("Testing accuracy:")
print(accuracy_score(Y_test, y_pred))

Testing accuracy:
0.5795


In [43]:
cm

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [47, 10, 17,  1,  0,  0,  0,  7,  3,  0,  0,  1,  0,  6,  0,  1,
         0,  0,  7,  0,  0],
       [14, 31,  8,  3,  0,  0,  3,  5,  7,  4,  1,  0,  1,  3,  0,  2,
         0,  0, 18,  0,  0],
       [ 0,  0, 53,  0,  0,  0,  0, 28,  3,  0,  1,  5,  4,  0,  0,  0,
         0,  0,  0,  6,  0],
       [ 0,  0,  2, 61, 16,  5,  4,  0,  1,  0,  0,  0,  2,  0,  0,  6,
         0,  0,  3,  0,  0],
       [ 2,  1,  3,  4, 67,  0,  5,  2,  3,  0,  5,  0,  0,  0,  0,  7,
         0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 98,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,
         0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0, 89,  2,  3,  0,  1,  0,  0,  0,  0,  4,
         0,  0,  0,  0,  0],
       [ 4,  0, 23,  0,  0,  0,  0, 45, 15,  3,  5,  2,  0,  0,  0,  0,
         0,  0,  1,  2,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  3, 76,  1,  0, 19,  0,  0,  0,  0,
         0,  0,  0,

In [44]:
#F1_score
from sklearn.metrics import f1_score
print("Testing F1-score")
print(f1_score(Y_test, y_pred, pos_label='positive', average='micro'))

Testing F1-score
0.5795


C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [47]:
#Precision
from sklearn.metrics import precision_score
print("Testing Precision:")
print(precision_score(Y_test, y_pred, pos_label='positive', average='micro'))
#Recall
from sklearn.metrics import recall_score
print("Testing Recall:")
print(recall_score(Y_test, y_pred, pos_label='positive', average='micro'))

Testing Precision:
0.5795
Testing Recall:
0.5795


C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [48]:
# save to JSON
my_model.save_model("model.json")
# save to text format
my_model.save_model("model.txt")

In [ ]:
X_train.shape